Learn GEE

https://developers.google.com/earth-engine/guides/image_visualization

In [10]:
if 'google.colab' in str(get_ipython()):
    from google.colab import userdata
    EE_PROJECT_ID = userdata.get('EE_PROJECT_ID') 
else:
    from dotenv import load_dotenv
    import os
    load_dotenv()  # take environment variables
    EE_PROJECT_ID = os.getenv('EE_PROJECT_ID')
    
# Set up GEE API
import ee
ee.Authenticate()
ee.Initialize(project=EE_PROJECT_ID)

# Import other libs
import geemap


## Masking

In [11]:
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')

# Create an NDWI image, define visualization parameters and display.
ndwi = image.normalizedDifference(['B3', 'B5'])
ndwi_viz = {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}

# Mask the non-watery parts of the image, where NDWI < 0.4.
ndwi_masked = ndwi.updateMask(ndwi.gte(0.4))

# Define a map centered on San Francisco Bay.
map_ndwi_masked = geemap.Map(center=[37.5010, -122.1899], zoom=10)

# Add the image layer to the map and display it.
map_ndwi_masked.add_layer(ndwi, ndwi_viz, 'NDWI')
map_ndwi_masked.add_layer(ndwi_masked, ndwi_viz, 'NDWI masked')
display(map_ndwi_masked)

Map(center=[37.501, -122.1899], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topr…

## Visualization images

In [ ]:
image_rgb = image.visualize(bands=['B5', 'B4', 'B3'], max=0.5)
ndwi_rgb = ndwi_masked.visualize(min=0.5, max=1, palette=['00FFFF', '0000FF'])

## Mosaicking

In [17]:
# Mosaic the visualization layers and display (or export).
mosaic = ee.ImageCollection([image_rgb, ndwi_rgb]).mosaic()

# Define a map centered on San Francisco Bay.
map_mosaic = geemap.Map(center=[37.5010, -122.1899], zoom=10)

# Add the image layer to the map and display it.
map_mosaic.add_layer(mosaic, None, 'mosaic')
display(map_mosaic)

Map(center=[37.501, -122.1899], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topr…

## Clipping

In [16]:
# Create a circle by drawing a 20000 meter buffer around a point.
roi = ee.Geometry.Point([-122.4481, 37.7599]).buffer(20000)
mosaic_clipped = mosaic.clip(roi)

# Define a map centered on San Francisco.
map_mosaic_clipped = geemap.Map(center=[37.7599, -122.4481], zoom=10)

# Add the image layer to the map and display it.
map_mosaic_clipped.add_layer(mosaic_clipped, None, 'mosaic clipped')
display(map_mosaic_clipped)

Map(center=[37.7599, -122.4481], controls=(WidgetControl(options=['position', 'transparent_bg'], position='top…

## Rendering categorical maps

In [22]:
# Load 2012 MODIS land cover and select the IGBP classification.
cover = ee.Image('MODIS/051/MCD12Q1/2012_01_01').select('Land_Cover_Type_1')

# Define a palette for the 18 distinct land cover classes.
igbp_palette = [
    'aec3d4',  # water
    '152106',
    '225129',
    '369b47',
    '30eb5b',
    '387242',  # forest
    '6a2325',
    'c3aa69',
    'b76031',
    'd9903d',
    '91af40',  # shrub, grass
    '111149',  # wetlands
    'cdb33b',  # croplands
    'cc0013',  # urban
    '33280d',  # crop mosaic
    'd7cdcc',  # snow and ice
    'f7e084',  # barren
    '6f6f6f',  # tundra
]

# Define a map centered on the United States.
map_palette = geemap.Map(center=[40.413, -99.229], zoom=5)

# Add the image layer to the map and display it. Specify the min and max labels
# and the color palette matching the labels.
map_palette.add_layer(
    cover, {'min': 0, 'max': 17, 'palette': igbp_palette}, 'IGBP classes'
)
display(map_palette)

Map(center=[40.413, -99.229], controls=(WidgetControl(options=['position', 'transparent_bg'], position='toprig…

## Thumbnail images

In [31]:
# Fetch a digital elevation model.
image = ee.Image('CGIAR/SRTM90_V4')

# Request a default thumbnail of the DEM with defined linear stretch.
# Set masked pixels (ocean) to 1000 so they map as gray.
thumbnail_1 = image.unmask(1000).getThumbURL({
    'min': 0,
    'max': 3000,
    'dimensions': 500,
})
print('Default extent:', thumbnail_1)

# Specify region by rectangle, define palette, set larger aspect dimension size.
thumbnail_2 = image.getThumbURL({
    'min': 0,
    'max': 3000,
    'palette': [
        '00A600',
        '63C600',
        'E6E600',
        'E9BD3A',
        'ECB176',
        'EFC2B3',
        'F2F2F2',
    ],
    'dimensions': 500,
    'region': ee.Geometry.Rectangle([-84.6, -55.9, -32.9, 15.7]),
})
print('Rectangle region and palette:', thumbnail_2)

# Specify region by a linear ring and set display CRS as Web Mercator.
thumbnail_3 = image.getThumbURL({
    'min': 0,
    'max': 3000,
    'palette': [
        '00A600',
        '63C600',
        'E6E600',
        'E9BD3A',
        'ECB176',
        'EFC2B3',
        'F2F2F2',
    ],
    'region': ee.Geometry.LinearRing(
        [[-84.6, 15.7], [-84.6, -55.9], [-32.9, -55.9]]
    ),
    'dimensions': 500,
    'crs': 'EPSG:3857',
})
print('Linear ring region and specified crs:', thumbnail_3)

from IPython.display import Image
display(Image(url=thumbnail_1))
display(Image(url=thumbnail_2))
display(Image(url=thumbnail_3))

Default extent: https://earthengine.googleapis.com/v1/projects/gen-lang-client-0038628791/thumbnails/3bbe407c4fb4d95e5a03d51383de3238-fc30cc8973d6eeac29ad4012a365f757:getPixels
Rectangle region and palette: https://earthengine.googleapis.com/v1/projects/gen-lang-client-0038628791/thumbnails/c422a98c58645c378ee7513b126ef150-43c1ad8b48f3b94e0a2f8e412e32ef90:getPixels
Linear ring region and specified crs: https://earthengine.googleapis.com/v1/projects/gen-lang-client-0038628791/thumbnails/05c351b5fe4884b734bd9a27b912a076-a8dcbeb670a038f398502123efc05aff:getPixels
